In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np 
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from scipy.stats import mode
from scipy.special import binom
from sklearn.metrics.pairwise import cosine_similarity
from time import time
import BII

# 1 Preprocess data

We are using the well known adult dataset.

In [2]:
train = pd.read_csv("../data/adult/adult.data",header=None)
test  = pd.read_csv("../data/adult/adult.test",header=None) 
columns = ["Age", "workclass", "fnlwgt", "education","education-num","marital-status", "occupation"
           ,"relationship", "race","sex","capital-gain","capital-loss", "hours-per-week", "native-country", "label"]
cat_columns = ["workclass", "education", "marital-status", "occupation"
           ,"relationship", "race","sex", "native-country", "label"]
cont_colums = [col for col in columns if col not in cat_columns]
train.columns, test.columns = columns, columns
#Dropping fnlwgt as it's not an actual feature but a weighting factor
train.drop("fnlwgt", axis=1, inplace=True)
test.drop("fnlwgt", axis=1, inplace=True)
columns.remove("fnlwgt")
cont_colums.remove("fnlwgt")

We encode the categorical variables

In [3]:
encoders = {}
train_transformed, test_transformed = pd.DataFrame(), pd.DataFrame()
for category in cat_columns:
    if category == "label":
        train_transformed[category] = train[category]
        test_transformed[category] = test[category]
        continue
    if category == "education":
        continue;
    encoder = OneHotEncoder()
    encoder.fit(np.asarray(train[category]).reshape(-1, 1))
    encoders[category] = encoder
    transformed_featue_train = pd.get_dummies(train[category],prefix=category)
    transformed_featue_test = pd.get_dummies(test[category],prefix=category)
    for column in transformed_featue_train.columns:
        train_transformed[column] = transformed_featue_train[column]
        try:
            test_transformed[column] = transformed_featue_test[column]
        except :
            test_transformed[column] = np.zeros(len(test_transformed))
for category in cont_colums:
    train_transformed[category] = train[category]
    test_transformed[category] = test[category]
# We treat education slightly differently as it is already encoded in "education-num", so we just use this encoding
#(it is ordered with respect to the level of education, while LabelEncoders are lexicographic) 
# The right way would be to drop the column but we leave it to show some axiomatic properties
train_transformed["education"] = train_transformed["education-num"]
test_transformed["education"] = test_transformed["education-num"]
cont_colums.append("education")
cat_columns.remove("education")

In [4]:
x_train, y_train = train_transformed.loc[:, train_transformed.columns != 'label'], train_transformed["label"]
x_test,  y_test  = test_transformed.loc[:, test_transformed.columns != 'label'], test_transformed["label"]
cat_columns.remove("label")
columns = [col for col in columns if col != "label"]
reordered_test = test[cont_colums + cat_columns]

In [5]:
[c for c in x_train.columns if "marital" in c]

['marital-status_ Divorced',
 'marital-status_ Married-AF-spouse',
 'marital-status_ Married-civ-spouse',
 'marital-status_ Married-spouse-absent',
 'marital-status_ Never-married',
 'marital-status_ Separated',
 'marital-status_ Widowed']

# 2 Train baseline classifier

For this table we trained a random forrest classifier

In [6]:
clf = RandomForestClassifier(max_depth=10, n_estimators=50)
clf = clf.fit(x_train, y_train)

In [7]:
clf.score(x_train, y_train), clf.score(x_test,  y_test)

(0.8626884923681705, 0.8589767213316135)

In [8]:
baseline = pd.DataFrame(data=np.zeros([1,len(x_train.columns)]),columns= x_train.columns)
for category in cont_colums:
    baseline[category] = np.median(x_train[category])

In [9]:
baseline

,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,marital-status_ Divorced,...,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,Age,education-num,capital-gain,capital-loss,hours-per-week,education
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,37.0,10.0,0.0,0.0,40.0,10.0


In [10]:
b_clf = BII.BanzhafModel(clf, np.asarray(baseline))

In [11]:
linear_points = np.arange(1,10000,10) 

In [12]:
test.iloc[linear_points]

,Age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
11,36,Federal-gov,Bachelors,13,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K
21,34,Private,Some-college,10,Never-married,Other-service,Own-child,Black,Female,0,0,35,United-States,<=50K
31,56,Self-emp-not-inc,11th,7,Widowed,Other-service,Unmarried,White,Female,0,0,50,United-States,<=50K
41,44,Self-emp-inc,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9951,64,State-gov,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
9961,24,Private,Bachelors,13,Never-married,Sales,Own-child,Black,Female,0,0,40,United-States,<=50K
9971,39,Self-emp-inc,Assoc-acdm,12,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,<=50K
9981,50,Self-emp-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K


In [13]:
onehot_columns = x_train.columns.to_list()

In [14]:
reordered_test_merge = reordered_test.drop("education",axis=1) 

## Binary explanations (Interactions)

In [15]:
number_samples = 10000

In [ ]:
%%time
n_features = len(cont_colums) + len(cat_columns)
bin_banzhaf_values = np.zeros([x_test.shape[0],n_features,n_features])
bin_shap_values = np.zeros([x_test.shape[0],n_features,n_features])
bin_qii_values = np.zeros([x_test.shape[0],n_features,n_features])
for i in linear_points:
    print(i, end=", ")
    test_point = np.asarray(x_test.iloc[i:i+1,:])
    bin_banzhaf_values[i] = BII.one_hot_binary_banzhaf(test_point,b_clf,cont_colums,
                                                              cat_columns,onehot_columns,
                                                              number_samples=number_samples)
    bin_shap_values[i] =  BII.one_hot_binary_shapley(test_point,b_clf,cont_colums,
                                                            cat_columns,onehot_columns,
                                                            number_samples=number_samples)
    bin_qii_values[i] =  BII.one_hot_binary_qii(test_point,b_clf,cont_colums,
                                                               cat_columns,onehot_columns)

1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391, 401, 411, 421, 431, 441, 451, 461, 471, 481, 491, 501, 511, 521, 531, 541, 551, 561, 571, 581, 591, 601, 611, 621, 631, 641, 651, 661, 671, 681, 691, 701, 711, 721, 731, 741, 751, 761, 771, 781, 791, 801, 811, 821, 831, 841, 851, 861, 871, 881, 891, 901, 911, 921, 931, 941, 951, 961, 971, 981, 991, 1001, 1011, 1021, 1031, 1041, 1051, 1061, 1071, 1081, 1091, 1101, 1111, 1121, 1131, 1141, 1151, 1161, 1171, 1181, 1191, 1201, 1211, 1221, 1231, 1241, 1251, 1261, 1271, 1281, 1291, 1301, 1311, 1321, 1331, 1341, 1351, 1361, 1371, 1381, 1391, 1401, 1411, 1421, 1431, 1441, 1451, 1461, 1471, 1481, 1491, 1501, 1511, 1521, 1531, 1541, 1551, 1561, 1571, 1581, 1591, 1601, 1611, 1621, 1631, 1641, 1651, 1661, 1671, 1681, 1691, 1701, 1711, 1721, 1731, 1741, 1751, 1761, 1771, 1781, 1791, 1801, 1811, 1821, 1831, 1841, 1

In [ ]:
%%time
n_features = len(cont_colums) + len(cat_columns)
merge_bin_banzhaf_values = np.zeros([x_test.shape[0],n_features-1,n_features-1])
merge_bin_shap_values = np.zeros([x_test.shape[0],n_features-1,n_features-1])
merge_bin_qii_values = np.zeros([x_test.shape[0],n_features-1,n_features-1])
for i in linear_points:
    print(i, end=", ")
    test_point = np.asarray(x_test.iloc[i:i+1,:])
    merge_bin_banzhaf_values[i] = BII.one_hot_binary_banzhaf_merge(test_point,b_clf,
                                                   cont_colums,cat_columns,
                                                   onehot_columns, [1,5], number_samples=number_samples)
    merge_bin_shap_values[i] =  BII.one_hot_binary_shapley_merge(test_point,b_clf,
                                                   cont_colums,cat_columns,
                                                   onehot_columns, [1,5], number_samples=number_samples)
    merge_bin_qii_values[i] =  BII.one_hot_binary_qii_merge(test_point,b_clf,
                                                   cont_colums,cat_columns,
                                                   onehot_columns,  [1,5])

In [ ]:
max_bin_ban = np.max(np.abs(merge_bin_banzhaf_values))
max_bin_sha = np.max(np.abs(merge_bin_shap_values))

In [ ]:
for i,col in  enumerate(reordered_test.columns):
    print(i,col)

In [ ]:
# We are looking for points where the results for Shapley are counter intuitve (with some margin)

In [ ]:
#Interaction with marriage_status
marriage_status_mistakes = linear_points[np.where(np.logical_and(np.logical_and(
    bin_shap_values[linear_points][:,1,7]/max_bin_sha < -0.01, 
    bin_shap_values[linear_points][:,5,7]/max_bin_sha > 0.01),
    np.abs(bin_shap_values[linear_points][:,1,7] +
    bin_shap_values[linear_points][:,5,7] )/max_bin_sha + 0.05 < np.abs(merge_bin_shap_values[linear_points][:,1,6]/max_bin_sha) )
)]
marriage_status_mistakes

In [ ]:
#Interaction with capital gain
capital_gain_mistakes = linear_points[np.where(np.logical_and(
    np.abs((bin_shap_values[linear_points][:,1,2] +
    bin_shap_values[linear_points][:,2,5] )/max_bin_sha)< 0.01,
    np.abs(merge_bin_shap_values[linear_points][:,1,2] /max_bin_sha)> 0.01)
)]
capital_gain_mistakes

In [ ]:
print("Shapley")
print("{},{},{},{},{}".format("number","educationNumCaptialGain","educationCaptialGain","mergeEducationCapitalGain","expectedMergeEducationCapitalGain"))
for j,i in enumerate(capital_gain_mistakes):
    print("{},{:.3f},{:.3f},{:.3f},{:.3f}".format(j, 
                                                  bin_shap_values[i][1,2]/max_bin_sha,
                                                  bin_shap_values[i][2,5]/max_bin_sha,
                                                  merge_bin_shap_values[i][1,2]/max_bin_sha,
                                                  (bin_shap_values[i][1,2]+bin_shap_values[i][2,5])/max_bin_sha))

In [ ]:
print("{},{},{},{},{}".format("number","educationNumMarriage","educationMarriage","mergeEducationMarriage","expectedMergeEducationMarriage"))
for j,i in enumerate(marriage_status_mistakes):
    print("{},{:.3f},{:.3f},{:.3f},{:.3f}".format(j, 
                                                  bin_shap_values[i][1,7]/max_bin_sha,
                                                  bin_shap_values[i][5,7]/max_bin_sha,
                                                  merge_bin_shap_values[i][1,6]/max_bin_sha,
                                                  (bin_shap_values[i][1,7]+bin_shap_values[i][5,7])/max_bin_sha))

### This is just for testing

In [ ]:
print("Banzhaf")
print("{},{},{},{},{}".format("number","educationNumCaptialGain","educationCaptialGain","mergeEducationCapitalGain","expectedMergeEducationCapitalGain"))
for j,i in enumerate(capital_gain_mistakes):
    print("{},{:.3f},{:.3f},{:.3f},{:.3f}".format(j, 
                                                  bin_banzhaf_values[i][1,2]/max_bin_ban,
                                                  bin_banzhaf_values[i][2,5]/max_bin_ban,
                                                  merge_bin_banzhaf_values[i][1,2]/max_bin_ban,
                                                  (bin_banzhaf_values[i][1,2]+bin_banzhaf_values[i][2,5])/max_bin_ban))

In [ ]:
print("{},{},{},{},{}".format("number","educationNumMarriage","educationMarriage","mergeEducationMarriage","expectedMergeEducationMarriage"))
for j,i in enumerate(marriage_status_mistakes):
    print("{},{:.3f},{:.3f},{:.3f},{:.3f}".format(j, 
                                                  bin_banzhaf_values[i][1,7]/max_bin_ban,
                                                  bin_banzhaf_values[i][5,7]/max_bin_ban,
                                                  merge_bin_banzhaf_values[i][1,6]/max_bin_ban,
                                                  (bin_banzhaf_values[i][1,7]+bin_banzhaf_values[i][5,7])/max_bin_ban))